# **genshin-toolkit 伤害计算器教程**
这个教程将会逐步指导你完成
- 角色、武器、圣遗物的配置
- buff、hit的编写
- 计算类的初始化与计算
- 高级应用

### **导入必须的库**
依赖顺序 
gtenum->artifact->damagecalc

In [1]:
#gtenum定义了基本类型、属性、伤害等枚举，用import * 的方式导入
from genshintoolkit.gtcommon.gtenum import *
#artifact.py定义了Artifact、ArtifactSet两个圣遗物类
from genshintoolkit.artifact import Artifact,ArtifactSet,ArtifcatStats
#damagecalc.py定义了Buff、hit等重要类，用import * 的方式导入
from genshintoolkit.damagecalc import *
#导入交互用控件，用于生成圣遗物数值
import ipywidgets as wd
from IPython.display import display

### **定义角色和武器**
这里以乘区齐全而且有属性转换的角色胡桃作为例子来解说

In [2]:
#攻击防御生命的白值，角色突破成长属性、副属性用一个以EnumStat为键的dict来提供
hutao=Character(base_atk=106.4,base_def=876,base_hp=15552,base_bonus={EnumStats.crit_dmg:0.384})
homa=Weapon(base_atk=608,substat={EnumStats.crit_dmg:0.662},effect=None)

### **设置圣遗物**
设置圣遗物有两种方式
- Artifact类
- stat dict

In [3]:
#直接输入数值完成圣遗物设置，其中main_stat和sub_stat都是stat dict
flower=Artifact(setname=EnumArtifactSet.CrimsonWitchOfFlames,
                piecename_cn='魔女花', #注释名不会影响结果，可以空串留空或者用喜欢的名字
                position=EnumArtifactPosition.flower,
                level=20,
                star=5,
                main_stat={EnumStats.hp_static:4780},
                sub_stats={
                    EnumStats.atk_percent:0.156,
                    EnumStats.crit_rate:0.039,
                    EnumStats.crit_dmg:0.078,
                    EnumStats.elemental_mastery:21
                })
print(flower.sub_stats)


{<EnumStats.atk_percent: 'atk_percent'>: 0.156, <EnumStats.crit_rate: 'crit_rate'>: 0.039, <EnumStats.crit_dmg: 'crit_dmg'>: 0.078, <EnumStats.elemental_mastery: 'elemental_mastery'>: 21}


Artifact库提供了一些工具函数用于生成圣遗物

In [4]:

#使用artifact库里的pull_substats函数随机抽取词条，接受的参数是主词条的枚举
from genshintoolkit.artifact import pull_substats
plume=Artifact(setname=EnumArtifactSet.CrimsonWitchOfFlames,
                piecename_cn='魔女毛', #注释名不会影响结果，可以空串留空或者用喜欢的名字
                position=EnumArtifactPosition.plume,
                level=20,
                star=5,
                main_stat={EnumStats.hp_static:4780},#不会检查属性的类别和数值，可以放心填写
                sub_stats=pull_substats(EnumStats.atk_static)
                )

#或者可以用pull_piece方法直接抽取一件，此处重复抽取直到抽到生命沙为止,此方法接收EnumAritifactSet作为参数，默认为Dummy
#position作为可选参数
from genshintoolkit.artifact import pull_piece
sands:Artifact
while True:
    sands=pull_piece(EnumArtifactSet.CrimsonWitchOfFlames,position=EnumArtifactPosition.sands)
    if sands.position==EnumArtifactPosition.sands:
        break

#根据词条数来生成圣遗物,输入的参数是四个词条的元组和强化元组的元组
from genshintoolkit.artifact import gen_artifact_bypoints
#需要的4个词条分别为大生命、暴击、暴伤、精通
globlet_stats=(EnumStats.hp_percent,EnumStats.crit_rate,EnumStats.cd,EnumStats.em)
#生命跳2次第二档1次第三档，暴击跳3次最高档，爆伤精通初始分别为第一、第二档
globlet_matrix=(
    (0,2,1,0),
    (0,0,0,3),
    (1,0,0,0),
    (0,1,0,0)
)
globlet=Artifact(setname=EnumArtifactSet.CrimsonWitchOfFlames,
                piecename_cn='魔女杯', #注释名不会影响结果，可以空串留空或者用喜欢的名字
                position=EnumArtifactPosition.globlet,
                level=20,
                star=5,
                main_stat={EnumStats.dmg_pyro:0.466},
                sub_stats=gen_artifact_bypoints(sub_stats=globlet_stats,matrix=globlet_matrix)
                )


In [5]:

#使用get_artifact_bypoints_avg方法，根据词条平均值来生成圣遗物
from genshintoolkit.artifact import gen_artifact_bypoints_avg
slider1=wd.IntSlider(value=0,min=1,max=5,step=1,description='hp%')
slider2=wd.IntSlider(value=0,min=1,max=5,step=1,description='crit rate')
slider3=wd.IntSlider(value=0,min=1,max=5,step=1,description='crit dmg')
slider4=wd.IntSlider(value=0,min=1,max=5,step=1,description='elem mastery')
circlet_sliders=(slider1,slider2,slider3,slider4)
for s in circlet_sliders:
    display(s)


IntSlider(value=1, description='hp%', max=5, min=1)

IntSlider(value=1, description='crit rate', max=5, min=1)

IntSlider(value=1, description='crit dmg', max=5, min=1)

IntSlider(value=1, description='elem mastery', max=5, min=1)

In [6]:

circlet_vector=(slider1.value,slider2.value,slider3.value,slider4.value)
print(circlet_vector)
circlet=Artifact(setname=EnumArtifactSet.CrimsonWitchOfFlames,
                piecename_cn='魔女头', #注释名不会影响结果，可以空串留空或者用喜欢的名字
                position=EnumArtifactPosition.circlet,
                level=20,
                star=5,
                main_stat={EnumStats.crit_rate:0.311},
                sub_stats=gen_artifact_bypoints_avg((EnumStats.hp_percent,EnumStats.crit_rate,EnumStats.cd,EnumStats.em),circlet_vector))
print(circlet.sub_stats)
                


(1, 1, 1, 1)
None


In [7]:

atf_set1=ArtifactSet(flower=flower,plume=plume,sands=sands,goblet=globlet,circlet=circlet)

#ArtifactSet内置unpack方法可以把圣遗物解包成stat dict形式，或者你可以直接编写stat dict并将它传给dmgcalc对象
#atf_statdict=ArtifactSet.unpack()
#print(atf_statdict)

### **编写buff**
此处使用班尼特、云堇、砂糖、万叶来介绍buff的编写方法

In [8]:
buff_bennnet_q=Buff(
    name='bennett_q',#会被作为检索的依据
    apply_to=ApplyTagsSet(
        type_action=[EnumAction.all,],#对普重下落等所有动作类型生效
        type_element=[EnumHitElementType.all,],#只对元素生效用elemental，全生效或物理生效用all，physical
        type_reaction=[EnumReaction.all,] #蒸发要同时写上vape_pryo和vape_hydro，融化同理
        #TODO:优化applyto条件
    ),
    target_stat=EnumStats.atk_static,#加成的类型是固定生命值
    value=(191+674)*1.39,#数值取90级班尼特雾切13级q
    required_stats=None,#不随被buff角色自身的属性变化
    calc_func=None,#同上
    is_snapshot=False, #不存在快照
    is_static=True #不随被buff角色自身的属性变化
)
#无条件buff可用预定义的无条件预设applytag_all

buff_hutao_e=Buff(
    name='hutao_e',
    apply_to=applytag_all,
    target_stat=EnumStats.atk_static,
    value=None,#随被buff角色数值变化的，value留成None
    required_stats=[EnumStats.hp_static,],#需要根据自身的生命值来计算加成数值
    calc_func=lambda x :x[EnumStats.hp_static]*0.0626,#程序会根据require_stats要求获取的面板数值返回相应的stat dict，calc_func要以statdict作为参数并返回数值
    is_snapshot=True, #在e开启的一瞬间锁面板，后续不再变化
    is_static=False #不随被buff角色自身的属性变化
    
)

#包含复合效果的技能，对每个数值的修改都要单独写成独立的buff
buff_mona_q_dmgbonus=Buff(
    name='mona_q_dmgbonus',
    apply_to=applytag_all,
    target_stat=EnumStats.dmg_bonus_all,
    value=0.54,
    required_stats=None,
    calc_func=None,
    is_snapshot=True,
    is_static=True
)

buff_mona_q_amping=Buff(
    name='mona_q_amping',
    apply_to=ApplyTagsSet(
        type_action=[EnumAction.all,],
        type_element=[EnumHitElementType.hydro,EnumHitElementType.pyro,EnumHitElementType.cryo],
        type_reaction=[EnumReaction.melt_cryo,EnumReaction.melt_pyro,EnumReaction.vape_hydro,EnumReaction.vape_pyro]
    ),
    target_stat=EnumStats.reaction_bonus_amping,
    value=0.15
)

#魔女二件套、魔女四件套

### **编写Hit**
Hit即角色做出一次攻击动作，hit对象内包含了伤害的动作类型（普攻、重击、下落），元素类型（元素、物理），反应类型（无，蒸发、融化），剧变反应，治疗等信息

In [ ]:
hit_hutao_q=Hit(
    type_action=EnumAction.burst,
    type_element=EnumHitElementType.pyro,
    multipliers=[
        Multiplier(  #第一个效果，造成攻击倍率为494%的蒸发伤害
            hit_result=EnumHitResult.amping_dmg,
            atk=4.94,
            hp=0,
            defn=0,
            em=0
        ),
        Multiplier(  #第二个效果，造成生命倍率为13.60%的治疗效果,倍率默认值为0，可不填
            hit_result=EnumHitResult.heal,
            atk=0,
            defn=0,
            hp=0.1360,
            em=0
        ),
        Multiplier(  #第三个效果，造成超载伤害，倍率留空
            hit_result=EnumHitResult.overload
        )
    ],
    durition=0, #目前无相关功能
    comment='' #对buff的注释
)